In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 18456



### conversions - from date shaped char

In [8]:
# date shaped char table
sas.submit(
f"""
data old_table;
    input date_column $;
    datalines;
202101
.
;
run;
""")

sas.sasdata("old_table","work").columnInfo()

,Member,Num,Variable,Type,Len,Pos
0,WORK.OLD_TABLE,1.0,date_column,Char,8.0,0.0


In [29]:
def make_sql(key_str):
    return f"""
title;
    
proc print data=old_table;
run;

proc sql;
    create table new_table as
    select
{key_str}
    from old_table;
quit;

proc print data=new_table;
run;
"""

In [24]:
# date shaped char -> date
s = make_sql("""
        CATS(date_column, '01') AS cats,
        INPUT(CATS(date_column, '01'), YYMMDD10.) AS new_date FORMAT=DATE9.,
        INPUT(IFC(date_column = ' ', ' ', CATS(date_column, '01')), YYMMDD10.) AS new_date2 FORMAT=DATE9.
""")
sas.submitLST(s, method="listonly")

In [26]:
# will throw error inside a data step
sas.submitLST("""
data _null_;
        set old_table;
        new_date = INPUT(CATS(date_column, '01'), YYMMDD10.);
        format new_date DATE9.;
run;
""")


64                                                         The SAS System                         Thursday, May 16, 2024 11:28:00 AM

655        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
655      !  graphics on / outputfmt=png;
656        
657        
658        data _null_;
659                set old_table;
660                new_date = INPUT(CATS(date_column, '01'), YYMMDD10.);
661                format new_date DATE9.;
662        run;
date_column=  new_date=. _ERROR_=1 _N_=2
663        
664        
665        
666        ods html5 (id=saspy_internal) close;ods listing;
667        

65                                                         The SAS System                         Thursday, May 16, 2024 11:28:00 AM

668        


In [34]:
# date shaped char -> date number (from left justified to right justified)
s = make_sql("""
        INPUT(date_column, 6.) AS date_number
""")
sas.submitLST(s, method="listonly")

Obs,date_column
1,202101
2,
Obs,date_number
1,202101
2,.


### conversions - from date shaped num

In [36]:
sas.submitLST(
f"""
data old_table;
    input date_column 10.;
    datalines;
202101
.
;
run;
""")
sas.sasdata("old_table","work").columnInfo()


82                                                         The SAS System                         Thursday, May 16, 2024 11:28:00 AM

882        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
882      !  graphics on / outputfmt=png;
883        
884        
885        data old_table;
886            input date_column 10.;
887            datalines;
890        ;
891        run;
892        
893        
894        
895        ods html5 (id=saspy_internal) close;ods listing;
896        

83                                                         The SAS System                         Thursday, May 16, 2024 11:28:00 AM

897        


,Member,Num,Variable,Type,Len,Pos
0,WORK.OLD_TABLE,1.0,date_column,Num,8.0,0.0


In [37]:
# date shaped number -> date shaped char
s = make_sql("""
        PUT(date_column, 6.) AS new_date_char
""")
sas.submitLST(s)

Obs,date_column
1,202101
2,.
Obs,new_date_char
1,202101
2,.


### arithmetic of date shaped numbers

#### via conversion to number

In [65]:
sas.submitLST(
f"""
/* Add x number of months */
data add_months;
    /* inputs */
    input date $6.;
    x = 4;

    /* process */
    year = input(substr(date,1,4), 4.);
    month = input(substr(date,5,2), 2.);
    month = month + x;
    if month > 12 then do;
        year = year + 1;
        month = month - 12;
    end;
    new_date = trim(put(year,4.) || put(month,z2.));
    new_date_num = input(new_date, 6.);

    datalines;
202409
;
run;

proc print data=add_months;
run;
""")

Obs,date,x,year,month,new_date,new_date_num
1,202409,4,2025,1,202501,202501


In [60]:
sas.submitLST(
f"""
data _null_;
    string1 = ' Hello ';
    string2 = ' World ';
    new_string = catx('', string1, string2);
    new_string2 = "!" || new_string || "!";
    new_string21 = trim(new_string) || "end";
    new_string22 = "!" || "Hello World" || "!";
    new_string3 = "!" || string1 || string2 || "!";

    put new_string;
    put new_string2;
    put new_string21;
    put new_string22;
    put new_string3;
run;
""")


147                                                        The SAS System                         Thursday, May 16, 2024 11:28:00 AM

1585       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
1585     !  graphics on / outputfmt=png;
1586       
1587       
1588       data _null_;
1589           string1 = ' Hello ';
1590           string2 = ' World ';
1591           new_string = catx('', string1, string2);
1592           new_string2 = "!" || new_string || "!";
1593           new_string21 = trim(new_string) || "end";
1594           new_string22 = "!" || "Hello World" || "!";
1595           new_string3 = "!" || string1 || string2 || "!";
1596       
1597           put new_string;
1598           put new_string2;
1599           put new_string21;
1600           put new_string22;
1601           put new_string3;
1602       run;
Hello World
!Hello World                                                                    

In [67]:
sas.submitLST(
f"""
/* Add y number of years */
data add_years;
    input date $6.;
    y = 2; /* number of years to add */

    /* process */
    year = input(substr(date,1,4), 4.);
    month = input(substr(date,5,2), 2.);
    year = year + y;
    new_date = put(year,4.) || put(month,z2.);
    new_date_num = input(new_date, 6.);
    
    datalines;
202409
;
run;

proc print data=add_years;
run;
""")

Obs,date,y,year,month,new_date,new_date_num
1,202409,2,2026,9,202609,202609


In [74]:
sas.submitLST(
f"""
/* Difference between two dates */
data date_diff;
    input date1 :$6. date2 :$6.;

    year1 = substr(date1,1,4);
    month1 = substr(date1,5,2);
    year2 = substr(date2,1,4);
    month2 = substr(date2,5,2);

    diff_years = year2 - year1;
    diff_months = month2 - month1;
    if diff_months < 0 then do;
        diff_years = diff_years - 1;
        diff_months = diff_months + 12;
    end;
    diff_months = diff_years * 12 + diff_months;
    
    datalines;
202409 202503
202409 202510
202409 202301 
202409 202310 
202409 202408
202409 202409
202409 202410
;
run;

proc print data=date_diff;
run;
""")

Obs,date1,date2,year1,month1,year2,month2,diff_years,diff_months
1,202409,202503,2024,09,2025,03,0,6
2,202409,202510,2024,09,2025,10,1,13
3,202409,202301,2024,09,2023,01,-2,-20
4,202409,202310,2024,09,2023,10,-1,-11
5,202409,202408,2024,09,2024,08,-1,-1
6,202409,202409,2024,09,2024,09,0,0
7,202409,202410,2024,09,2024,10,0,1


#### via conversion to sas date

In [87]:
sas.submitLST(
f"""
data inputs;
    date_char = '202409';
    do x = -20 to 20 by 5;
        output;
    end;
run;

proc print data=inputs;
run;
""")

Obs,date_char,x
1,202409,-20
2,202409,-15
3,202409,-10
4,202409,-5
5,202409,0
6,202409,5
7,202409,10
8,202409,15
9,202409,20


In [85]:
x = 4
sas.submitLST(
f"""
/* Add x number of months */
proc sql;
    select 
        date_char,
        x,
        intnx('month', input(date_char, yymmn6.), x) as new_date format=yymmn6.,
        put(intnx('month', input(date_char, yymmn6.), x), yymmn6.) as new_date_char
    from inputs;
    ;
quit;
""")

date_char,x,new_date,new_date_char
202409,-20,202301,202301
202409,-15,202306,202306
202409,-10,202311,202311
202409,-5,202404,202404
202409,0,202409,202409
202409,5,202502,202502
202409,10,202507,202507
202409,15,202512,202512
202409,20,202605,202605


In [89]:
sas.submitLST(
f"""
/* Add y number of years */
proc sql;
    select 
        date_char,
        x,
        intnx('year', input(date_char, yymmn6.), x) as new_date format=yymmn6.,
        put(intnx('year', input(date_char, yymmn6.), x), yymmn6.) as new_date_char
    from inputs;
    ;
quit;
""")

date_char,x,new_date,new_date_char
202409,-20,200401,200401
202409,-15,200901,200901
202409,-10,201401,201401
202409,-5,201901,201901
202409,0,202401,202401
202409,5,202901,202901
202409,10,203401,203401
202409,15,203901,203901
202409,20,204401,204401


In [90]:
sas.submitLST(
f"""
data inputs;
    input date1_char :$6. date2_char :$6.;
    datalines;
202409 202503
202409 202510
202409 202301
;
run;

/* Difference between two dates */
proc sql;
    select 
        date1_char,
        date2_char,
        intck('month', input(date1_char, yymmn6.), input(date2_char, yymmn6.)) as diff_months
    from inputs
    ;
quit;
""")

date1_char,date2_char,diff_months
202409,202503,6
202409,202510,13
202409,202301,-20


### reference table of skeleton dates

In [102]:
start_date = '202409'
end_date = '202503'

sas.submitLST(
f"""
/* Difference between two dates */
data _null_;
    start_date = input('{start_date}', yymmn6.);
    end_date = input('{end_date}', yymmn6.);
    diff_months = intck('month', start_date, end_date);
    call symputx('diff_months', diff_months);
run;

%put &diff_months;

data reference_table;
    /* generate a reference table with all months between start and end date */
    do i = 0 to &diff_months;
        start_date = input('{start_date}', yymmn6.);
        format start_date yymmn6.;
        date = intnx('month', start_date, i);
        format date yymmn6.;
        date_char = put(date, yymmn6.);
        date_num = input(date_char, 6.);
        output;
    end;
run;

proc print data=reference_table;
run;
""", method="listonly")

Obs,i,start_date,date,date_char,date_num
1,0,202409,202409,202409,202409
2,1,202409,202410,202410,202410
3,2,202409,202411,202411,202411
4,3,202409,202412,202412,202412
5,4,202409,202501,202501,202501
6,5,202409,202502,202502,202502
7,6,202409,202503,202503,202503


### comparison to datestring

In [ ]:
sas.submitLST(f"""
    proc print data=sashelp.air;
        where date <= "01mar1949"d;
    run;
""")

Obs,DATE,AIR
1,JAN49,112
2,FEB49,118
3,MAR49,132


### input string as date

In [8]:
# forms that start with yy can be parsed via anydtdte10 informat.
sas.submitLST(f"""
    data df;
        input d1 anydtdte10.;
        format d1 ddmmyy10.;
        datalines;
2025-02-13
1993-02-13
1993/02/13
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/1993
3,13/02/1993


In [9]:
# a common form is ddmmyy10.
sas.submitLST(f"""
    data df;
        input d1 ddmmyy10.;
        format d1 ddmmyy10.;
        datalines;
13/02/2025
13-02-2025
13.02.2025
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/2025
3,13/02/2025


In [10]:
# the core date9 format is ddmmmyyyy
sas.submitLST(f"""
    data df;
        input d1 date9.;
        format d1 date9.;
        datalines;
13feb2025
13FEB2025
13feb40
        ;
    run;
              
    proc print data=df;
    run;
""")

sas.sasdata('df').columnInfo()

Obs,d1
1,13FEB2025
2,13FEB2025
3,13FEB1940


,Member,Num,Variable,Type,Len,Pos,Format
0,WORK.DF,1.0,d1,Num,8.0,0.0,DATE9.
